# Welcome. 
This is a program that gives you visual matches between galaxies found in SDSS data and spectral lines fit in the GUI

In [1]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from pandas.tools.plotting import table
import json
from IPython.display import display
import re
from astropy.cosmology import WMAP7 as cosmo
from astropy import units as u
from astropy import constants as const
import matplotlib.pyplot as plt
from astropy.table import QTable
import sympy as sp
import os
from os import listdir 
from astropy.table import Table, Column
from matplotlib.backends.backend_pdf import PdfPages
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

# Please type your path below. The following cell requires your attention

In [2]:
your_path = "/Users/dusty736/Documents/499-QSO"  #place your own path to the quasar file!!

for file in os.listdir(your_path):
    if file.endswith(".json"):
        print(file)
print('\x1b[1;35m'+ "copy from above and paste your json file below" + '\x1b[0m')
name = input('type file name here:  ') 
name_path = your_path + "//" + name
with open(name_path) as data_file:    
    data = json.load(data_file)

J1016+4706_werksquad.json
copy from above and paste your json file below
type file name here:  "J1016+4706_werksquad.json"


## Section 1: Finding Your Quasar

This section runs through a list of quasars and matches the quasar based on the name of your quasar file from the spectral analysis GUI. The assumption made is that your quasar file has the the quasar name in the file. 

Eg. "J1241+5721_formal.json"

If not, there will be a line for you to place the name of the quasar (Eg. "JXXXX+XXXX")

In [3]:
raw_quasar_radec = open("qsoinfo.txt", "r").read()  # this line requires that you downloaded qsoinfo.txt

In [4]:
revised_quasar_radec = raw_quasar_radec.splitlines(0)[1:]
qs0 = np.array([])
qso_long = np.array([])
gmag = np.array([])
ra = np.array([])
dec = np.array([])
qso_redshift = np.array([])

for i in range(0 , len(revised_quasar_radec)):
    line = revised_quasar_radec[i].split(" ")
    useful_list = np.array([])
    for t in range(0 , len(line)):
        if line[t] != "":
            useful_list = np.append(useful_list , line[t])    
    qs0 = np.append(qs0 , useful_list[0])
    qso_long = np.append(qso_long , useful_list[1])
    ra = np.append(ra , useful_list[2])
    dec = np.append(dec , useful_list[3])
    qso_redshift = np.append(qso_redshift , useful_list[4])
    gmag = np.append(gmag , useful_list[5])
    
qsolist = Table([qs0,qso_long,gmag,ra,dec, qso_redshift], 
      names=("qso","qso_long","gmag","ra","dec" , "qso_redshift"))

### WARNING: 
If your quasar file name doesn't have the name in the title or is not in the qsolist, the following cell will not work. Skip it and manually input the quasar values in the next cell

In [5]:
name_edit = name.split("_")
for i in range(0 , len(name_edit)):
    n = name_edit[i]
    if "J" == n[0] and (n[1] and n[2] and n[5] and n[9]) in "0123456789+-":
        name_compare = name_edit[i]
        break
    else:
        name_compare = "Failure, type in quasar data in cell below"


for i in range(0 , len(qsolist["qso"])):
    if name_compare == qsolist["qso"][i]:
        qso_index = qsolist[i]

print(qso_index)

   qso           qso_long       gmag     ra      dec    qso_redshift
---------- ------------------- ----- --------- -------- ------------
J1016+4706 J101622.60+470643.3 17.12 154.09418 47.11204        0.822


## Section 2: Using the Query

This section looks at the sdss query and puts the data of galaxies near the ra , dec value of the quasar.

In [6]:
ra_init = float(qso_index["ra"])
dec_init = float(qso_index["dec"])
redshift_init = float(qso_index["qso_redshift"])

In [7]:
quasar_ra = ra_init * u.deg
quasar_dec = dec_init * u.deg
quasar_redshift = redshift_init

quasar_ra_1d = quasar_ra / u.deg
quasar_dec_1d = quasar_dec / u.deg

#Note, these are the parameters for searching los 
separation = .010 #of .010 redshift between an SDSS object and an absorber from the spectrum in the quasar
proximity = .5  # this is distance in Mpc from the LOS of the quasar.
spec_range = .3
zcc2_range = .05
ranges = zcc2_range    # set to spec_range if desired

ra_low = str(ra_init - ranges)
ra_high = str(ra_init + ranges)
dec_low = str(dec_init - ranges)
dec_high = str(dec_init + ranges)

In [ ]:
print "Copy the following into the SQL search at http://cas.sdss.org/dr7/en/tools/search/sql.asp:"
print "select  photoz.z, photoz.zErr, photoz2.photozcc2 , photoz2.photozerrcc2 ,"
print " photoz2.photozd1, u, err_u, ra, dec, extinction_r"
print "from photoz , photoz2 , PhotoObjAll"
print "where  PhotoObjAll.objID = photoz.objID AND"
print "PhotoObjAll.objID = photoz2.objID AND"
print "PhotoObjAll.ra between " + ra_low + " and " + ra_high + " AND"
print "PhotoObjAll.dec between """ + dec_low + " and " + dec_high

In [59]:
qso = pd.read_csv('quasar_data.csv')

In [60]:
z = qso['z']
zErr = qso['zErr']
photozcc2 = qso['photozcc2']
photozerrcc2 = qso['photozerrcc2']
photozd1 = qso['photozd1']
u_xx = qso['u']
err_u = qso['err_u']
ra = qso['ra']
dec = qso['dec']
extinction_r = qso['extinction_r']
    
query_data = Table([z,zErr,photozcc2,photozerrcc2,photozd1,u_xx,err_u,ra,dec,extinction_r], 
      names=("z","zErr","photozcc2","photozerrcc2","photozd1","u","err_u","ra","dec","extinction_r"))

## Section 3: Analyzing the Query Data

This section is a series of functions applied to the data file from the SDSS query. The result is a new table with new data derived through functions from old data

In [61]:
arcsecperrad = (u.arcsec / u.rad).si
def real_angular_diam_dist(redshift):
    value = cosmo.kpc_proper_per_arcmin(redshift)
    return(value.to(u.kpc / u.arcsec))
    

def angular_separation(ra1,dec1,ra2,dec2):
    dela = ra1 * u.deg- ra2 
    deldec = dec1 * u.deg - dec2
    delthet = np.sqrt((dela * np.cos(deldec))**2 + deldec ** 2)
    return(delthet.to(u.arcsec))
    
def angular_separation_notdegrees(ra1h, ra1m, ra1s, dec1d, dec1m, dec1s, ra2h, ra2m, ra2s, dec2d, dec2m, dec2s):
    dela = ra1h * u.hourangle + ra1m * u.arcmin + ra1s * u.arcsec - (ra2h * u.hourangle + ra2m * u.arcmin + ra2s * u.arcsec)
    deldec = dec1d * u.deg + dec1m * u.arcmin + dec1s * u.arcsec - (dec2d * u.deg + dec2m * u.arcmin + dec2s * u.arcsec)
    print(deldec)
    deladeg = dela.to(u.deg)
    deldecdeg = deldec.to(u.deg)
    delthet = np.sqrt((deladeg*np.cos(deldecdeg))**2 + deldecdeg ** 2)
    print(delthet)

def distance_from_los(ang_sep, ang_diam_dist):
    final = ((ang_sep/u.arcsec) * (ang_diam_dist* u.arcsec/u.kpc)) 
    final = final/1000
    return final

In [62]:
zcc2redshift = query_data['photozcc2','ra','dec']

photozcc2
---------
 0.124031
 0.308945
 0.058533
 0.220184
 0.261006
 0.315481
 0.102208
 0.129848
 0.181621
 0.167019
      ...
 0.403012
 0.192559
 0.466429
 0.324505
 0.541691
 0.557892
 0.381728
 0.653606
 0.550947
 0.377976
 0.225507
Length = 89 rows


In [63]:
zcc2redshift['angdiam'] = real_angular_diam_dist(zcc2redshift['photozcc2'])

zcc2redshift['angsep'] = angular_separation(zcc2redshift['ra'],zcc2redshift['dec'],quasar_ra,quasar_dec)
zcc2redshift['dist_from_los'] = distance_from_los(zcc2redshift['angsep'], zcc2redshift['angdiam'])
zcc2redshift["dist_from_los"].unit = u.Mpc
condensed_table = zcc2redshift['photozcc2', 'ra', 'dec', 'angsep', 'dist_from_los']


condensed_table.sort('photozcc2')
# condensed_table

## Section 4: Analyzing the JSON Spectrum file

This takes all the data in the JSON file and and puts the fittings into various lists such as: graphponents_filter as well as 
(strong_redshift , weak_redshift , all_redshift , metal_abs). 

In [64]:
radshift = []
ra = []
dec= []

In [65]:
name_list = ['HI', 'HeI', 'AlII', 'AlIII', 'ArVI', 'CI', 'CII', 'CII*', 'CIII', 'CIV', 'CaII', 'CaVIII',
             'CaX', 'ClII', 'FeII', 'FeIII', 'MgI', 'MgII', 'MgX', 'NI', 'NII', 'NIII',
             'NIV', 'NV', 'NaI', 'NeII', 'NeIII', 'NeIV', 'NeVI', 'NeVII', 'NeVIII', 'NiII', 'OI', 
             'OII', 'OIII', 'OIV', 'OV', 'OVI', 'OVII', 'OVIII', 'PII', 'PIII', 'PV', 'SII', 'SIII', 
             'SIV', 'SV', 'SVI', 'SiII', 'SiII*', 'SiIII', 'SiIV', 'TiII', 'ZnII']

components = []
graphponents = []
systems=['z0.00000_MW']
for cmp in data["cmps"]:
    cmp_dict=data["cmps"][str(cmp)]
    cmp_data=''
    if cmp[-2:]=="HI":
        systems.append(str(cmp))
    cmp_data+=str(cmp)+';'
    cmp_data+=str(cmp_dict["Reliability"])+';'
    if cmp_dict["Comment"]=='':
        systems.append(str(cmp))
        cmp_data+='None;'
    else:
        cmp_data+=str(cmp_dict["Comment"])+';'
    cmp_data+=str(cmp_dict["Nfit"])+';'
    cmp_data+=str(cmp_dict["bfit"])+';'
    components.append(cmp_data)
    graphponents.append(str(cmp))


comp_broken =[]
for x in range(0 , len(components)):
    comp_broken.append(components[x].split(";"))

graphponents_filter = []   
for filter_ in name_list: 
    graphponents_filter.append(len([pos for pos, s in enumerate(graphponents) if filter_ in s]))


In [66]:
new_data = []
name = []
rating = []
comment = []
Nc = []
bc = []
raw_z = []
strong_abs = []
strong_redshift =[]
has_metals = []
weak_abs = []
weak_redshift = []
all_redshift = []
all_abs = []
metal_abs = []

for component in components:
    cmp_info = component.split(';')
    #print(cmp_info)
    if component[1]=='-':
        z_simple = component[2:8]
    else:
        z_simple = component[1:7]
    condition1 , condition2 , condition3 , condition4 = True , True , True , True
    
    
    # condition 1 , what element do you want to find?
    if "HI" in cmp_info[0]:
        condition1 = True
    else:
        
        condition1 = False

    if float(cmp_info[3]) > 10:
        condition2 = True
    else:
        condition2 = False
    
    if "" in cmp_info[2]:
        condition3 = True
    else:
        condition3 = False
        
    conditions = condition1 and condition2 and condition3 and condition4
   
    if conditions == True:
        raw_z.append(float(z_simple))
        name.append(cmp_info[0])
        rating.append(cmp_info[1])
        comment.append("{0:.60s}".format(cmp_info[2]))    #change 60s to larger number for longer comments
        Nc.append(float("{0:.6s}".format(cmp_info[3])))
        bc.append(float("{0:.6s}".format(cmp_info[4])))

    if float(cmp_info[3]) > 13.8:
        condition5 = True
    else:
        condition5 = False

    if float(cmp_info[3]) > 13.0 and float(cmp_info[3]) <= 13.8  :
        condition6 = True
    else:
        condition6 = False
  
    if float(cmp_info[3]) > 10.0 :
        condition7 = True
    else:
        condition7 = False    
    
    if condition5 and condition1== True:
        strong_abs.append(float("{0:.6s}".format(cmp_info[3])))
        strong_redshift.append(float(z_simple))
        
    if condition6 and condition1== True:
        weak_abs.append(float("{0:.6s}".format(cmp_info[3])))
        weak_redshift.append(float(z_simple))
        
    if condition7 and condition1== True:
        all_abs.append(float("{0:.6s}".format(cmp_info[3])))
        all_redshift.append(float(z_simple))
        
    if condition1 == False: 
        metal_abs.append(float(z_simple))

## Section 5: Creating and Outputting Graphs

This section compares data from the SDSS query (Section 3) with data from the JSON spectrum file (Section 4) to find matches between real galaxies and Lymann Alpha fits.

In [67]:
workingarray1 = []
workingarray2 = []
workingarray3 = []
workingarray4 = []
workingarray5 = []
quantity_less_comparison = condensed_table 
quantity_less_comparison['dist_from_los'] = quantity_less_comparison['dist_from_los'] / u.Mpc 

for Idx1,Val1 in enumerate(condensed_table['photozcc2']):
    for Idx2,Val2 in enumerate(strong_redshift):
        difference = abs(condensed_table['photozcc2'][Idx1] - strong_redshift[Idx2])
        if difference < separation and condensed_table['dist_from_los'][Idx1] < proximity: 
            workingarray1.append(condensed_table['photozcc2'][Idx1])
            workingarray2.append(float(quantity_less_comparison['dist_from_los'][Idx1]))
            workingarray3.append(strong_redshift[Idx2])
            workingarray4.append(condensed_table['ra'][Idx1])
            workingarray5.append(condensed_table['dec'][Idx1])

comparison_table= Table([workingarray1, workingarray2, workingarray3, \
workingarray4, workingarray5], names=('photozcc2', 'dist_from_los', \
'strong_redshift','ra','dec'), meta={'name': 'comparisontable'})

comparison_table["delta_z"] = comparison_table["photozcc2"] - comparison_table["strong_redshift"]
comparison_table = comparison_table["photozcc2" , "strong_redshift" , "delta_z" ,
                                    "dist_from_los" , "ra" , "dec"]
scale = len(comparison_table)

figtable , ax = plt.subplots(1,1)
figtable.set_size_inches(16, 8)
clust_data = comparison_table
collabel=('photoz2', 'dist_from_los', 'hir','delta_z','ra','dec')
table = ax.table(cellText=clust_data,colLabels=collabel,loc='top' , cellLoc='left', 
                 bbox=[-0.1, 0.2, 0.7, 1.1] , 
                 colWidths = [.9,1.6,.7,1,1.4,1.3])

ax.axis("off")
table.auto_set_font_size(False)
table.set_fontsize(14)
#ax.text(.5 , 1 , "Matches Data", horizontalalignment='center' , fontsize = 20)
ax.table?

#plt.show()

In [68]:
%%capture
bin_redshift = condensed_table['dist_from_los']
bin_sep = 0.150
bins = [0, bin_sep]
hist, bin_edges = np.histogram(bin_redshift, bins=bins)
bins = 15
hist, bin_edges = np.histogram(bin_redshift, bins=bins)


#######
figtable , ax = plt.subplots(1,1)
figtable.set_size_inches(16, 16)
ax.set_title("Matches list between galaxy and HI lines" , fontsize = 20)
clust_data = comparison_table
collabel=('phtzcc2', 'HI_z', 'delta_z', 'dist_from_los', 'ra','dec')
table = ax.table(cellText=clust_data,colLabels=collabel,loc='middle' , cellLoc='left', 
                 bbox=[0.12, 0.5 - 0.012 * scale, 0.7, 0.023 * scale] , 
                 colWidths = [1.1,.9,1.1,1.8,1.6,1.6])
ax.axis("off")
table.auto_set_font_size(False)
table.set_fontsize(12)
#ax.text(.5 , 1 , "Matches Data", horizontalalignment='center' , fontsize = 20)

########
fig1, ax = plt.subplots(1,1)
ax.set_title('number of absorbers. Data sorted in bins of '+str(bin_sep))
ax.set_xlabel('distance from LOS in MPC')
ax.set_ylabel('count')
ax.hist(bin_redshift, bins, edgecolor = 'black');

#######
fig2, ax = plt.subplots(1,1)
ax.plot(condensed_table['photozcc2'], condensed_table['dist_from_los'], 'ro',color = 'blue');
ax.set_title('distance from los in MPC versus redshift')
ax.set_xlabel('redshift')
ax.set_ylabel('distance from los in MPC')

valer = 0
valery = 0

#######
fig3, walker = plt.subplots(1,1)
fig3.set_size_inches(15,2)
walker.set_xlim(0,quasar_redshift + .05)
walker.vlines(quasar_redshift, -1.8, -.2, color='green', linewidth=3)   
walker.vlines(strong_redshift, -1.8, -.2, color='red', linewidth=1)
walker.vlines(quasar_redshift, -1,1,color='none',linewidth=0)
walker.vlines(weak_redshift, -1.8, -.2, color='gray', linewidth=.5)
walker.vlines(metal_abs,-3,-2,color='black',linewidth = .5)
walker.set_title('Green = RED = strong absorbers, Blue = Objects in LOS from SDSS, Purple = Quasar, GRAY = weak absorbers, BLACK = metals')
walker.plot(condensed_table['photozcc2'], np.zeros_like(condensed_table['photozcc2']) + valer, '|', markersize=15,);


baler = 0
balery = 0

#########
fig4, two_walker = plt.subplots(1,1)
fig4.set_size_inches(15,15)
two_walker.set_xlim(0,quasar_redshift + .05)
two_walker.vlines(quasar_redshift, 0, 2, color='green', linewidth=1)   
two_walker.vlines(strong_redshift, 0,  .25, color='red', linewidth=1)
two_walker.vlines(quasar_redshift,0,  .25,color='none',linewidth=0)
two_walker.vlines(weak_redshift,0,  .25, color='gray', linewidth=.5)
two_walker.vlines(metal_abs,-.25,0,color='black',linewidth = .5)
two_walker.set_title('Red = strong absorbers, Purple = strong absorbers with strong sdss fits, Blue = Objects in LOS from SDSS, Green= Quasar, GRAY = weak absorbers, BLACK = metals')
two_walker.plot(condensed_table['photozcc2'], condensed_table['dist_from_los'],marker = 'o',linestyle='none',markersize = 6);



########
fig5, ax = plt.subplots(1,1)
ax.plot(comparison_table['photozcc2'], comparison_table['dist_from_los'], 'ro',color = 'purple');
ax.set_title('distance from los in MPC versus redshift')
ax.set_xlabel('redshift')
ax.set_ylabel('distance from los in MPC')



#######
fig6, three_walker = plt.subplots(1,1)
fig6.set_size_inches(15,15)
three_walker.set_xlabel("Redshift")
three_walker.set_ylabel("Mpc")
three_walker.set_xlim(0,quasar_redshift + .05)
three_walker.vlines(quasar_redshift, 0, 2, color='green', linewidth=1)   
three_walker.vlines(comparison_table['strong_redshift'], -.1,  comparison_table['dist_from_los'] + .1, color='m', linewidth=2)
three_walker.vlines(quasar_redshift,0,  .25,color='none',linewidth=0)
three_walker.vlines(metal_abs,-.25,0,color='black',linewidth = .5)
three_walker.set_title('Purple = strong absorbers with good fits || \n Green = Quasar, BLACK = metals')
three_walker.plot(comparison_table['photozcc2'], comparison_table['dist_from_los'],marker = '*',linestyle='none'
                  ,markersize = 10,color = '#040273');


######
fig7, two_walker = plt.subplots(1,1)
fig7.set_size_inches(15,15)
two_walker.set_xlabel("Redshift")
two_walker.set_ylabel("Mpc")
two_walker.set_xlim(0,quasar_redshift + .05)
two_walker.vlines(quasar_redshift, 0, 2, color='green', linewidth=1)   
two_walker.vlines(strong_redshift, 0,  .25, color='red', linewidth=1)
two_walker.vlines(quasar_redshift,0,  .25,color='none',linewidth=0)
two_walker.vlines(weak_redshift,0,  .25, color='gray', linewidth=.5)
two_walker.vlines(comparison_table['strong_redshift'], 0,  .25 , color='m', linewidth=2)
two_walker.vlines(metal_abs,-.25,0,color='black',linewidth = .5)
two_walker.set_title('RED = strong absorbers \n PURPLE = strong absorbers with strong sdss fits \n \
BLUE = Objects in LOS from SDSS \n GREEN= Quasar \n GRAY = weak absorbers \n BLACK = metals \n STAR = matching galaxy')
two_walker.plot(condensed_table['photozcc2'], condensed_table['dist_from_los'],marker = 'o',linestyle='none',markersize = 6);
two_walker.plot(comparison_table['photozcc2'], comparison_table['dist_from_los'],marker = '*',linestyle='none',
                markersize = 10, color = '#040273');


######
fig8, ax = plt.subplots(1,1)
ax.set_title('histogram of all absorbers based on redshift')
ax.hist(all_redshift, bins=10, facecolor='MediumOrchid');


######
fig9, ax = plt.subplots(1,1)
ax.set_title('historgram of strong absorbers based on redshift')
ax.hist(strong_redshift, bins=10, facecolor='MediumOrchid');


######
fig10, ax = plt.subplots(1,1)
ax.set_title('histogram of weak absorbers based on redshift')
ax.hist(weak_redshift, bins=10, facecolor='MediumOrchid');

######
fig11 , ax = plt.subplots(1,1)
fig11.set_size_inches(15,8)
ax.text(-2.8, 0.2, 'HI={0:d}'.format(graphponents_filter[0]), color='red', fontsize=18)
amt_ticks = np.arange(len(graphponents_filter))
ax.bar(amt_ticks,[0] + graphponents_filter[1:]);
ax.set_title("Quantity per Metal" , fontsize = 20)
ax.set_xticks(np.arange(len(graphponents_filter)))
ax.set_xticklabels((name_list) , rotation = "vertical")


valer = 0
valery = 0
#######
fig12, walker = plt.subplots(1,1)
fig12.set_size_inches(15,2)
walker.set_xlim(0,quasar_redshift + .05)
walker.vlines(quasar_redshift, -1, 1, color='green', linewidth=3)   
walker.vlines(strong_redshift,-1, 1, color='red', linewidth=1)
walker.vlines(quasar_redshift,-1, 1,color='none',linewidth=0)
walker.vlines(weak_redshift, -1, 1, color='gray', linewidth=.5)
walker.vlines(metal_abs,-2,-1,color='black',linewidth = .5)
plt.title('Green = RED = strong absorbers, Purple = Quasar, GRAY = weak absorbers, BLACK = metals')


note = """{0} matches were found.
""".format(len(comparison_table))

header = "Graphical Analysis of QSO {0:s}".format(name_compare)

left, width = .25, .5
bottom, height = .25, .5
right = left + width
top = bottom + height
fig_text , ax = plt.subplots(1,1)
fig_text.set_size_inches(10,13)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.axis('off')
ax.text(0.5*(left+right), (bottom+top), header,
        horizontalalignment='center',
        verticalalignment='center',
        fontsize=24, color='purple',
        transform=ax.transAxes)
ax.text(0.5*(left+right), 0.5*(bottom+top), note,
        horizontalalignment='center',
        verticalalignment='center',
        fontsize=12, color='black',
        transform=ax.transAxes)


pp = PdfPages(name_compare + '_Analysis.pdf')
pp.savefig(fig_text)
pp.savefig(figtable)

pp.savefig(fig7) # main full graph with matches
pp.savefig(fig6) # main graph; matches only
pp.savefig(fig2) # MPC vs. redshift of galaxies
pp.savefig(fig5) # MPC vs. redshift of galaxy matches
pp.savefig(fig1) # number of absorbers
pp.savefig(fig8) # general histograms
pp.savefig(fig9) # general histograms
pp.savefig(fig10) # general histograms
pp.savefig(fig11) # quantity metal bar plot
pp.savefig(fig3) # quasar analysis only
pp.savefig(fig12) # quasar analysis only
pp.savefig(fig4) # main full graph without matches

pp.close()